In [ ]:
# some useful mysklearn package import statements and reloads
import importlib

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

# uncomment once you paste your mypytable.py into mysklearn package
import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyNaiveBayesClassifier, MyDecisionTreeClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

In [ ]:
print("hi")

## Load and clean the data

In [ ]:
mytable = MyPyTable()
fName = os.path.join("InputData", "yelp_academic_dataset_business.json")
dataset = mytable.load_from_json_file(fName, 30000)
dataset.remove_rows_with_missing_values()

id_ = dataset.get_column('business_id')
review_count = dataset.get_column('review_count')
attributes = dataset.get_column('attributes')

mytable2 = MyPyTable()
dataset2 = mytable2.load_from_json_file('yelp_academic_dataset_tip.json', 30000)

id_2 = dataset2.get_column('business_id')
user_id = dataset2.get_column('user_id')
text = dataset2.get_column('text')

new_id = []
new_user_id = []
new_text = []
for id in id_:
    if id in id_2:
        new_id.append(id)
        index = id_2.index(id)
        new_user_id.append(user_id[index])
        new_text.append(text[index])

new_id = new_id[:1500]
new_user_id = new_user_id[:1500]
new_text = new_text[:1500]

new_rev_count = []
new_attributes = []
for id in new_id:
    index = id_.index(id)
    new_rev_count.append(review_count[index])
    new_attributes.append(attributes[index])

mytable3 = MyPyTable()
dataset3 = mytable3.load_from_json_file('yelp_academic_dataset_user.json', 250000)

user_id = dataset3.get_column('user_id')
fans = dataset3.get_column('fans')
compliment_plain = dataset3.get_column('compliment_plain')
friends = dataset3.get_column('friends')
useful = dataset3.get_column('useful')

new_fans = []
new_comp_plain = []
new_friends = []
new_useful = []
for id in new_user_id: 
    if id in user_id:
        index = user_id.index(id)
        new_fans.append(fans[index])
        new_comp_plain.append(compliment_plain[index])
        new_friends.append(friends[index])
        new_useful.append(useful[index])

new_dataset = [[a, b, c, d, e, f, g] for a, b, c, d, e, f, g in zip(new_rev_count, new_attributes, new_text, new_fans, new_comp_plain, new_friends, new_useful)]

columns = ['Review Count', 'Attributes', 'Review Length', 'Fans', 'Compliment Plain', 'Friends', 'Useful']
final_dataset = MyPyTable(columns, new_dataset)

final_dataset.replace_missing_values_with_column_average('Review Count')
final_dataset.replace_missing_values_with_column_average('Attributes')
final_dataset.replace_missing_values_with_column_average('Review Length')
final_dataset.replace_missing_values_with_column_average('Fans')
final_dataset.replace_missing_values_with_column_average('Compliment Plain')
final_dataset.replace_missing_values_with_column_average('Friends')
final_dataset.replace_missing_values_with_column_average('Useful')

final_dataset.remove_rows_with_missing_values()

final_dataset.get_shape()

## Stratified k-fold Accuracy Check

In [ ]:
#==================================
#      Get stratified training    =
#      and testing sets           =
#==================================
k = 10
X, y = myutils.split_x_y_train(dataset.data)
x_train, x_test, y_train, y_test = myevaluation.train_test_split(X, y, shuffle=True)

# Get the training and testing folds
train_folds, test_folds = myevaluation.stratified_kfold_cross_validation(X, y, k)

# Get the traininga and testing sets from the folds
x_train, y_train, x_test, y_test = myutils.get_values_from_folds(X, y, train_folds, test_folds)


#==================================
#      Naive Bayes Classifier     =
#==================================
myNB = MyNaiveBayesClassifier()
myNB.fit(x_train, y_train)

# Compare predicted with actual
y_predict = myNB.predict(x_test)
count = 0
for i in range(len(y_predict)):
    if (y_predict[i] == y_test[i]):
        count = count + 1;

# Calculate accuracy and error
accuracy = count / len(y_predict)
error = (len(y_predict) - count) / len(y_predict)

print("Naive Bayes: accuracy =", accuracy, "error =", error)

#==================================
#     Decision Tree Classifier    =
#==================================
myDT = MyDecisionTreeClassifier()
myDT.fit(x_train, y_train)

# Compare predicted with actual
y_predict = myDT.predict(x_test)
count = 0
for i in range(len(y_predict)):
    if (y_predict[i] == y_test[i]):
        count = count + 1

# Calculate accuracy and error
accuracy = count / len(y_predict)
error = (len(y_predict) - count) / len(y_predict)

print("Decision Tree: accuracy =", accuracy, "error =", error)

#==================================
#     Random Forest Classifier    =
#==================================
print("Random Forest: accuracy =", "tbd", "error =", "tbd")

## Confusion Matrices

In [ ]:
# Get the x values to perform the matrix over
x = ['yes', 'no']

actual = []
nb_predicted = []
for i in range(len(y_test)):
    actual.append(y_test[i])
    nb_predicted.append(y_predict[i])

# Get the matrix from stratified
nb_matrix = myevaluation.confusion_matrix(actual, nb_predicted, x)

# Make the table header and calculate the statistics
table_header = ['Survived', "Yes", "No", 'Total', 'Recognition (%)']
complete_matrix = myutils.calc_matrix_stats(nb_matrix, True)

print()
print('Naive Bayes Result')
myutils.print_tabulate(complete_matrix, table_header)

In [ ]:
# Get the x values to perform the matrix over
x = ['yes', 'no']

actual = []
dt_predicted = []
for i in range(len(y_test)):
    actual.append(y_test[i])
    dt_predicted.append(y_predict[i])

# Get the matrix from stratified
dt_matrix = myevaluation.confusion_matrix(actual, dt_predicted, x)

# Make the table header and calculate the statistics
table_header = ['Survived', "Yes", "No", 'Total', 'Recognition (%)']
complete_matrix = myutils.calc_matrix_stats(dt_matrix, True)

print()
print('Decision Tree Result')
myutils.print_tabulate(complete_matrix, table_header)